In [1]:
import numpy as np
from tensorflow.keras.datasets import fashion_mnist
from tensorflow.keras.utils import to_categorical

# 1. Wczytanie danych
(train_X, train_y), (test_X, test_y) = fashion_mnist.load_data()

# 2. Normalizacja i reshape
train_X = train_X.reshape(-1, 28*28) / 255.0
test_X = test_X.reshape(-1, 28*28) / 255.0

train_y_onehot = to_categorical(train_y, 10)
test_y_onehot = to_categorical(test_y, 10)

# 3. Inicjalizacja wag
input_size = 784
hidden_size = 128
output_size = 10

np.random.seed(42)
W1 = np.random.randn(input_size, hidden_size) * 0.01
b1 = np.zeros(hidden_size)
W2 = np.random.randn(hidden_size, output_size) * 0.01
b2 = np.zeros(output_size)

# 4. Funkcje aktywacji i softmax
def relu(x):
    return np.maximum(0, x)

def relu_deriv(x):
    return (x > 0).astype(float)

def softmax(x):
    exp_x = np.exp(x - np.max(x, axis=1, keepdims=True))
    return exp_x / np.sum(exp_x, axis=1, keepdims=True)

# 5. Hyperparametry
lr = 0.1
epochs = 5
batch_size = 64

# 6. Pętla treningowa (SGD)
for epoch in range(epochs):
    # Losowe tasowanie
    permutation = np.random.permutation(train_X.shape[0])
    train_X = train_X[permutation]
    train_y_onehot = train_y_onehot[permutation]

    for i in range(0, train_X.shape[0], batch_size):
        X_batch = train_X[i:i+batch_size]
        y_batch = train_y_onehot[i:i+batch_size]

        # Forward
        z1 = X_batch.dot(W1) + b1
        a1 = relu(z1)
        z2 = a1.dot(W2) + b2
        a2 = softmax(z2)

        # Loss gradient (cross-entropy)
        dz2 = a2 - y_batch

        # Backpropagation
        dW2 = a1.T.dot(dz2) / batch_size
        db2 = np.sum(dz2, axis=0) / batch_size

        da1 = dz2.dot(W2.T)
        dz1 = da1 * relu_deriv(z1)

        dW1 = X_batch.T.dot(dz1) / batch_size
        db1 = np.sum(dz1, axis=0) / batch_size

        # Update weights
        W2 -= lr * dW2
        b2 -= lr * db2
        W1 -= lr * dW1
        b1 -= lr * db1

    # Proste metryki na koniec epoki
    z1 = train_X.dot(W1) + b1
    a1 = relu(z1)
    z2 = a1.dot(W2) + b2
    a2 = softmax(z2)
    preds = np.argmax(a2, axis=1)
    acc = np.mean(preds == train_y)
    print(f"Epoch {epoch+1}, Train accuracy: {acc:.4f}")


Epoch 1, Train accuracy: 0.1013
Epoch 2, Train accuracy: 0.0993
Epoch 3, Train accuracy: 0.1008
Epoch 4, Train accuracy: 0.0995
Epoch 5, Train accuracy: 0.1014


In [1]:
import tensorflow as tf
from tensorflow.keras.datasets import fashion_mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense
from tensorflow.keras.utils import to_categorical

# Ustaw GPU (jeśli jest) lub wymuś CPU
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"  # Odkomentuj jeśli problemem jest GPU

# Dane
(train_X, train_y), (test_X, test_y) = fashion_mnist.load_data()
train_X, test_X = train_X / 255.0, test_X / 255.0

train_y_cat = to_categorical(train_y, 10)
test_y_cat = to_categorical(test_y, 10)

# Model
model = Sequential([
    Flatten(input_shape=(28, 28)),
    Dense(64, activation='relu'),     # Mniejsza warstwa
    Dense(10, activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Trening
model.fit(train_X, train_y_cat, epochs=3, batch_size=32, validation_split=0.1)

# Test
loss, acc = model.evaluate(test_X, test_y_cat)
print(f"Test accuracy: {acc:.4f}")


Epoch 1/3
1688/1688 [==============================] - 5s 3ms/step - loss: 0.5274 - accuracy: 0.8177 - val_loss: 0.4369 - val_accuracy: 0.8403
Epoch 2/3
1688/1688 [==============================] - 4s 2ms/step - loss: 0.4000 - accuracy: 0.8572 - val_loss: 0.3943 - val_accuracy: 0.8582
Epoch 3/3
313/313 [==============================] - 1s 2ms/step - loss: 0.4002 - accuracy: 0.8576
Test accuracy: 0.8576
